## Setup

In [1]:
import pandas as pd
import numpy as np
import json
import os
from google.colab import drive

In [2]:
drive.mount('/content/drive')

current_directory = os.getcwd()

chris_directory = "/content/drive/Othercomputers/My_MacBook_Pro/Documents/Research/CADAS"
os.chdir(chris_directory)

current_directory = os.getcwd()

print(current_directory)

Mounted at /content/drive
/content/drive/Othercomputers/My_MacBook_Pro/Documents/Research/CADAS


Sociodemographic

In [9]:
Socio = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Sociodemographic")  # replace with your file path
    .loc[lambda df: df['Field_Type'] == 'Option', ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Socio['Special_Info'] = Socio['Special_Info'].str.lower()
Socio['Name'] = Socio['Name'].str.lower()
Socio['Special_Info'] = Socio['Special_Info'].str.replace(' required', '', case=False)

columns_to_process = ['Field_Type', 'Name', 'Special_Info']

for col in columns_to_process:
    Socio[col] = Socio[col].astype(str).str.strip()

Socio['Name_Cap'] = Socio['Name'].str.upper()

split_special_info = Socio['Special_Info'].str.split(',', expand=True)

split_special_info.columns = range(1, len(split_special_info.columns)+1)
split_special_info = split_special_info.applymap(lambda x: x.strip() if x is not None else None)

Socio = pd.concat([Socio, split_special_info], axis=1)

Socio = Socio.replace({None: np.nan})

for i in range(1, 15):
    Socio[f'new_column{i}'] = Socio.apply(lambda row: str(f'{Socio.columns[i+3]} "{row[i]}" '), axis=1).astype(str)

for i in range(1, 15):
    Socio[f'new_column{i}'] = (
       Socio[f'new_column{i}']
        .apply(lambda x: '8 "no responde"' if 'no responde' in str(x).lower() else x)
        .apply(lambda x: '9 "no sabe"' if 'no sabe' in str(x).lower() else x)
        .apply(lambda x: '0 "no"' if '"no"' in str(x).lower() else x)
        .apply(lambda x: '1 "si"' if '"si"' in str(x).lower() else x)
    )

Socio['new_combined_column'] = 'label define ' + Socio['Name_Cap'].astype(str) + ' .a"."' + ' '

for i in range(1, 15):
    Socio['new_combined_column'] += Socio.apply(lambda row: row[f"new_column{i}"] if 'nan' not in str(row[f"new_column{i}"]).lower() else '', axis=1)

Socio["capture"] = Socio.apply(lambda row: f'capture confirm numeric variable {row["Name_Cap"]}\nif !_rc'+"{", axis=1)
Socio["convert"] = Socio.apply(lambda row: f'     tostring {row["Name_Cap"]}, replace\n'+"}", axis=1)
Socio['encode_column'] = Socio.apply(lambda row: f'encode {row["Name_Cap"]}, gen({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Socio["rename"] = Socio.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)

Socio['combined_master_columns'] = Socio.apply(lambda row: f'{row["rename"]}\n\n{row["capture"]}\n{row["convert"]}\n\n{row["new_combined_column"]}\n\n{row["encode_column"]}', axis=1)

Socio_file = 'Recode_Files/Socio_code.txt'
with open(Socio_file, 'w') as f:
    for code_line in Socio['combined_master_columns']:
        f.write(f"{code_line}\n\n")

#dealing with checkboxes
Socio = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Sociodemographic")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Checkbox') | (df['Field_Type'] == 'Checkbox'), ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Socio['Name_Cap'] = Socio['Name']
Socio['Name'] = Socio['Name'].str.lower()

Socio["rename"] = Socio.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)
Socio["label"] = Socio.apply(lambda row: f'label define {row["Name_Cap"]} 0 "false" 1 "true"', axis=1)
Socio["encoded"] = Socio.apply(lambda row: f'encode {row["Name_Cap"]}, gen ({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Socio["drop"] = Socio.apply(lambda row: f'drop {row["Name_Cap"]}', axis=1)
Socio["checked_unchecked"] = Socio.apply(lambda row: f'label values {row["Name"]} checkbox_label', axis=1)
Socio['combined_master_columns'] = Socio.apply(lambda row: f'{row["rename"]}\n\n{row["label"]}\n\n{row["encoded"]}\n\n{row["drop"]}', axis=1)

Socio_file = 'Recode_Files/Socio_code_checks.txt'
with open(Socio_file, 'w') as f:
    # Write each code line from the "combined_master_columns" column
    for code_line in Socio['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Socio_file = 'Recode_Files/Socio_code_checks_renamed.txt'
with open(Socio_file, 'w') as f:
    # Write the additional text at the top of the file
    f.write('label define checkbox_label 0 "unchecked" 1 "checked", replace\n\n')

    # Write each code line from the "combined_master_columns" column
    for code_line in Socio['checked_unchecked']:
        f.write(f"{code_line}\n\n")

Socio.head()

,Field_Type,Name,Special_Info,Name_Cap,rename,label,encoded,drop,checked_unchecked,combined_master_columns
0,Checkbox,s_7_7a,NaN,S_7_7a,rename s_7_7a (S_7_7a),"label define S_7_7a 0 ""false"" 1 ""true""","encode S_7_7a, gen (s_7_7a) label (S_7_7a)",drop S_7_7a,label values s_7_7a checkbox_label,rename s_7_7a (S_7_7a)\n\nlabel define S_7_7a ...
1,Checkbox,s_7_7b,NaN,S_7_7b,rename s_7_7b (S_7_7b),"label define S_7_7b 0 ""false"" 1 ""true""","encode S_7_7b, gen (s_7_7b) label (S_7_7b)",drop S_7_7b,label values s_7_7b checkbox_label,rename s_7_7b (S_7_7b)\n\nlabel define S_7_7b ...
2,Checkbox,s_7_7c,NaN,S_7_7c,rename s_7_7c (S_7_7c),"label define S_7_7c 0 ""false"" 1 ""true""","encode S_7_7c, gen (s_7_7c) label (S_7_7c)",drop S_7_7c,label values s_7_7c checkbox_label,rename s_7_7c (S_7_7c)\n\nlabel define S_7_7c ...
3,Checkbox,s_7_7d,NaN,S_7_7d,rename s_7_7d (S_7_7d),"label define S_7_7d 0 ""false"" 1 ""true""","encode S_7_7d, gen (s_7_7d) label (S_7_7d)",drop S_7_7d,label values s_7_7d checkbox_label,rename s_7_7d (S_7_7d)\n\nlabel define S_7_7d ...
4,Checkbox,s_7_7e,NaN,S_7_7e,rename s_7_7e (S_7_7e),"label define S_7_7e 0 ""false"" 1 ""true""","encode S_7_7e, gen (s_7_7e) label (S_7_7e)",drop S_7_7e,label values s_7_7e checkbox_label,rename s_7_7e (S_7_7e)\n\nlabel define S_7_7e ...


forcing text fields as text fields

In [ ]:
Socio = (
    pd.read_excel("CADAS_DD_V3.xlsx", engine='openpyxl', sheet_name="Sociodemographic")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Text') | (df['Field_Type'] == 'Legal Values'), ['Field_Type', 'Name']]
    .reset_index(drop=True)
)

Socio['Name'] = Socio['Name'].str.lower()

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Socio[col] = Socio[col].astype(str).str.strip()

Socio['Name_Cap'] = Socio['Name'].str.upper()

Socio["capture"] = Socio.apply(lambda row: f'capture confirm numeric variable {row["Name"]}\nif !_rc'+"{", axis=1)
Socio["convert"] = Socio.apply(lambda row: f'     tostring {row["Name"]}, replace\n'+"}", axis=1)

Socio['combined_master_columns'] = Socio.apply(lambda row: f'{row["capture"]}\n{row["convert"]}', axis=1)

Socio_file = 'Recode_Files/Socio_text.txt'
with open(Socio_file, 'w') as f:
    for code_line in Socio['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Socio.head()

,Field_Type,Name,Name_Cap,capture,convert,combined_master_columns
0,Legal Values,s_particid2,S_PARTICID2,capture confirm numeric variable s_particid2\n...,"tostring s_particid2, replace\n}",capture confirm numeric variable s_particid2\n...
1,Text,s_deviceid1,S_DEVICEID1,capture confirm numeric variable s_deviceid1\n...,"tostring s_deviceid1, replace\n}",capture confirm numeric variable s_deviceid1\n...
2,Legal Values,s_1_2,S_1_2,capture confirm numeric variable s_1_2\nif !_rc{,"tostring s_1_2, replace\n}",capture confirm numeric variable s_1_2\nif !_r...
3,Legal Values,s_2_16,S_2_16,capture confirm numeric variable s_2_16\nif !_rc{,"tostring s_2_16, replace\n}",capture confirm numeric variable s_2_16\nif !_...
4,Legal Values,s_3_3_p,S_3_3_P,capture confirm numeric variable s_3_3_p\nif !...,"tostring s_3_3_p, replace\n}",capture confirm numeric variable s_3_3_p\nif !...


Adding variable labels

In [ ]:
Socio = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Sociodemographic") #everything except label/title, group, Time
    .loc[lambda df: ~df['Field_Type'].isin(['Label/Title', 'Group', 'Time','Date'])]
    .reset_index(drop=True)
)

Socio['Name'] = Socio['Name'].str.lower()

columns_to_process = ['Field_Type', 'Name']
Socio['Prompt'] = Socio['Prompt'].replace(r'[\'"]', '', regex=True)


for col in columns_to_process:
    Socio[col] = Socio[col].astype(str).str.strip()

Socio["label"] = Socio.apply(lambda row: f'label variable {row["Name"]} "{row["Prompt"]}"', axis=1)

Socio_file = 'Recode_Files/Socio_variable_labels.txt'
with open(Socio_file, 'w') as f:
    for code_line in Socio['label']:
        f.write(f"{code_line}\n\n")

name_list = ' '.join(Socio['Name'])

# Add the word 'order' at the beginning of the string
name_list = 'order ' + name_list

# Write the string to a text file
with open('Recode_Files/Socio_Names.txt', 'w') as f:
    f.write(name_list)


Socio.head()


,Page Position,Page Name,Tab Index,Prompt,Field_Type,Name,Variable Type,Format,Special_Info,label
0,0,Inro and Background,4,Número del entrevistador:,Number,s_interid,Number,NaN,Required,"label variable s_interid ""Número del entrevist..."
1,0,Inro and Background,5,Número de la casa:,Number,s_houseid,Number,NaN,Read Only,"label variable s_houseid ""Número de la casa:"""
2,0,Inro and Background,6,Número del participante:,Number,s_particid,Number,NaN,Read Only,"label variable s_particid ""Número del particip..."
3,0,Inro and Background,7,Número del conglomerado:,Number,s_clustid,Number,NaN,Read Only,"label variable s_clustid ""Número del conglomer..."
4,0,Inro and Background,8,Pais,Number,s_country,Number,NaN,Read Only,"label variable s_country ""Pais"""


Physical Exam

change to 0 base

In [10]:
Phys = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Physical_Exam")  # replace with your file path
    .loc[lambda df: df['Field_Type'] == 'Option', ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Phys['Special_Info'] = Phys['Special_Info'].str.lower()
Phys['Name'] = Phys['Name'].str.lower()
Phys['Special_Info'] = Phys['Special_Info'].str.replace(' required', '', case=False)

columns_to_process = ['Field_Type', 'Name', 'Special_Info']

for col in columns_to_process:
    Phys[col] = Phys[col].astype(str).str.strip()

Phys['Name_Cap'] = Phys['Name'].str.upper()

split_special_info = Phys['Special_Info'].str.split(',', expand=True)

split_special_info.columns = range(1, len(split_special_info.columns)+1)
split_special_info = split_special_info.applymap(lambda x: x.strip() if x is not None else None)

Phys = pd.concat([Phys, split_special_info], axis=1)

Phys = Phys.replace({None: np.nan})

for i in range(1, 9):
    Phys[f'new_column{i}'] = Phys.apply(lambda row: str(f'{Phys.columns[i+3]} "{row[i]}" '), axis=1).astype(str)

for i in range(1, 9):
    Phys[f'new_column{i}'] = (
       Phys[f'new_column{i}']
        .apply(lambda x: '8 "no responde"' if 'no responde' in str(x).lower() else x)
        .apply(lambda x: '9 "no sabe"' if 'no sabe' in str(x).lower() else x)
        .apply(lambda x: '0 "no"' if '"no"' in str(x).lower() else x)
        .apply(lambda x: '1 "si"' if '"si"' in str(x).lower() else x)
    )

Phys['new_combined_column'] = 'label define ' + Phys['Name_Cap'].astype(str) + ' .a"."' + ' '

for i in range(1, 9):
    Phys['new_combined_column'] += Phys.apply(lambda row: row[f"new_column{i}"] if 'nan' not in str(row[f"new_column{i}"]).lower() else '', axis=1)

Phys["capture"] = Phys.apply(lambda row: f'capture confirm numeric variable {row["Name_Cap"]}\nif !_rc'+"{", axis=1)
Phys["convert"] = Phys.apply(lambda row: f'     tostring {row["Name_Cap"]}, replace\n'+"}", axis=1)
Phys['encode_column'] = Phys.apply(lambda row: f'encode {row["Name_Cap"]}, gen({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Phys["rename"] = Phys.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)

Phys['combined_master_columns'] = Phys.apply(lambda row: f'{row["rename"]}\n\n{row["capture"]}\n{row["convert"]}\n\n{row["new_combined_column"]}\n\n{row["encode_column"]}', axis=1)


Phys_file = 'Recode_Files/Phys_code.txt'
with open(Phys_file, 'w') as f:
    for code_line in Phys['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Phys = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Physical_Exam")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Checkbox') | (df['Field_Type'] == 'Checkbox'), ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Phys['Name_Cap'] = Phys['Name']
Phys['Name'] = Phys['Name'].str.lower()

Phys["rename"] = Phys.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)
Phys["label"] = Phys.apply(lambda row: f'label define {row["Name_Cap"]} 0 "False" 1 "True"', axis=1)
Phys["encoded"] = Phys.apply(lambda row: f'encode {row["Name_Cap"]}, gen ({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Phys["drop"] = Phys.apply(lambda row: f'drop {row["Name_Cap"]}', axis=1)
Phys["checked_unchecked"] = Phys.apply(lambda row: f'label values {row["Name"]} checkbox_label', axis=1)
Phys['combined_master_columns'] = Phys.apply(lambda row: f'{row["rename"]}\n\n{row["label"]}\n\n{row["encoded"]}\n\n{row["drop"]}', axis=1)

Phys_file = 'Recode_Files/Phys_code_checks.txt'
with open(Phys_file, 'w') as f:
    for code_line in Phys['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Phys_file = 'Recode_Files/Phys_code_checks_renamed.txt'
with open(Phys_file, 'w') as f:
    # Write the additional text at the top of the file
    f.write('label define checkbox_label 0 "unchecked" 1 "checked", replace\n\n')

    # Write each code line from the "combined_master_columns" column
    for code_line in Phys['checked_unchecked']:
        f.write(f"{code_line}\n\n")

Phys.head()

,Field_Type,Name,Special_Info,Name_Cap,rename,label,encoded,drop,checked_unchecked,combined_master_columns


In [ ]:
Phys = (
    pd.read_excel("CADAS_DD_V3.xlsx", engine='openpyxl', sheet_name="Physical_Exam")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Text') | (df['Field_Type'] == 'Legal Values'), ['Field_Type', 'Name']]
    .reset_index(drop=True)
)

Phys['Name'] = Phys['Name'].str.lower()

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Phys[col] = Phys[col].astype(str).str.strip()

Phys['Name_Cap'] = Phys['Name'].str.upper()

Phys["capture"] = Phys.apply(lambda row: f'capture confirm numeric variable {row["Name"]}\nif !_rc'+"{", axis=1)
Phys["convert"] = Phys.apply(lambda row: f'     tostring {row["Name"]}, replace\n'+"}", axis=1)

Phys['combined_master_columns'] = Phys.apply(lambda row: f'{row["capture"]}\n{row["convert"]}', axis=1)

Phys_file = 'Recode_Files/Phys_text.txt'
with open(Phys_file, 'w') as f:
    for code_line in Phys['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Phys.head()


,Field_Type,Name,Name_Cap,capture,convert,combined_master_columns
0,Text,p_deviceid1,P_DEVICEID1,capture confirm numeric variable p_deviceid1\n...,"tostring p_deviceid1, replace\n}",capture confirm numeric variable p_deviceid1\n...
1,Legal Values,p_particid2,P_PARTICID2,capture confirm numeric variable p_particid2\n...,"tostring p_particid2, replace\n}",capture confirm numeric variable p_particid2\n...
2,Text,p_deviceid2,P_DEVICEID2,capture confirm numeric variable p_deviceid2\n...,"tostring p_deviceid2, replace\n}",capture confirm numeric variable p_deviceid2\n...


In [ ]:
Phys = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Physical_Exam") #everything except label/title, group, Time
    .loc[lambda df: ~df['Field_Type'].isin(['Label/Title', 'Group', 'Time','Date'])]
    .reset_index(drop=True)
)

Phys['Name'] = Phys['Name'].str.lower()
Phys['Prompt'] = Phys['Prompt'].replace(r'[\'"]', '', regex=True)


columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Phys[col] = Phys[col].astype(str).str.strip()

Phys["label"] = Phys.apply(lambda row: f'label variable {row["Name"]} "{row["Prompt"]}"', axis=1)

Phys_file = 'Recode_Files/Phys_variable_labels.txt'
with open(Phys_file, 'w') as f:
    for code_line in Phys['label']:
        f.write(f"{code_line}\n\n")

name_list = ' '.join(Phys['Name'])

# Add the word 'order' at the beginning of the string
name_list = 'order ' + name_list

# Write the string to a text file
with open('Recode_Files/Phys_Names.txt', 'w') as f:
    f.write(name_list)

Phys.head()


,Page Position,Page Name,Tab Index,Prompt,Field_Type,Name,Variable Type,Format,Special_Info,label
0,0,Intro and Background,1,Device ID:,Text,p_deviceid1,Text,NaN,Read Only,"label variable p_deviceid1 ""Device ID:"""
1,0,Intro and Background,5,Número del entrevistador:,Number,p_interid,Number,NaN,NaN,"label variable p_interid ""Número del entrevist..."
2,0,Intro and Background,6,Número del cluster:,Number,p_clustid,Number,NaN,Read Only,"label variable p_clustid ""Número del cluster:"""
3,0,Intro and Background,7,Número de la casa:,Number,p_houseid,Number,NaN,Read Only,"label variable p_houseid ""Número de la casa:"""
4,0,Intro and Background,8,Número del participante:,Number,p_particid,Number,NaN,Read Only,"label variable p_particid ""Número del particip..."


Cognitive

In [11]:
Cog = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Cognitive")  # replace with your file path
    .loc[lambda df: df['Field_Type'] == 'Option', ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Cog['Special_Info'] = Cog['Special_Info'].str.lower()
Cog['Name'] = Cog['Name'].str.lower()
Cog['Special_Info'] = Cog['Special_Info'].str.replace(' required', '', case=False)

columns_to_process = ['Field_Type', 'Name', 'Special_Info']

for col in columns_to_process:
    Cog[col] = Cog[col].astype(str).str.strip()

Cog['Name_Cap'] = Cog['Name'].str.upper()

split_special_info = Cog['Special_Info'].str.split(',', expand=True)

split_special_info.columns = range(0, len(split_special_info.columns))
split_special_info = split_special_info.applymap(lambda x: x.strip() if x is not None else None)

Cog = pd.concat([Cog, split_special_info], axis=1)

Cog = Cog.replace({None: np.nan})

for i in range(0, 6):
    Cog[f'new_column{i}'] = Cog.apply(lambda row: str(f'{Cog.columns[i+4]} "{row[i]}" '), axis=1).astype(str)

for i in range(0, 6):
    Cog[f'new_column{i}'] = (
        Cog[f'new_column{i}']
        .apply(lambda x: '8 "no responde"' if 'no responde' in str(x).lower() else x)
        .apply(lambda x: '9 "no sabe"' if 'no sabe' in str(x).lower() else x)
        .apply(lambda x: '5 "no pudo (anafalbeto)"' if "no pudo (anafalbeto)" in str(x).lower() else x)
        .apply(lambda x: '6 "no pudo (limitación física)"' if "no pudo (limitación física)" in str(x).lower() else x)
        .apply(lambda x: '7 "rehúsa"' if "rehúsa" in str(x).lower() else x)
        .apply(lambda x: '1 "correcto"' if '"correcto"' in str(x).lower() else x)
        .apply(lambda x: '0 "incorrecto"' if "incorrecto" in str(x).lower() else x)

    )

Cog['new_combined_column'] = 'label define ' + Cog['Name_Cap'].astype(str) + ' .a"."' + ' '

for i in range(0, 6):
    Cog['new_combined_column'] += Cog.apply(lambda row: row[f"new_column{i}"] if 'nan' not in str(row[f"new_column{i}"]).lower() else '', axis=1)

Cog["capture"] = Cog.apply(lambda row: f'capture confirm numeric variable {row["Name_Cap"]}\nif !_rc'+"{", axis=1)
Cog["convert"] = Cog.apply(lambda row: f'     tostring {row["Name_Cap"]}, replace\n'+"}", axis=1)
Cog['encode_column'] = Cog.apply(lambda row: f'encode {row["Name_Cap"]}, gen({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Cog["rename"] = Cog.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)

Cog['combined_master_columns'] = Cog.apply(lambda row: f'{row["rename"]}\n\n{row["capture"]}\n{row["convert"]}\n\n{row["new_combined_column"]}\n\n{row["encode_column"]}', axis=1)

Cog_file = 'Recode_Files/Cog_code.txt'
with open(Cog_file, 'w') as f:
    for code_line in Cog['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Cog = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Cognitive")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Checkbox') | (df['Field_Type'] == 'Checkbox'), ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Cog['Name_Cap'] = Cog['Name']
Cog['Name'] = Cog['Name'].str.lower()

Cog["rename"] = Cog.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)
Cog["label"] = Cog.apply(lambda row: f'label define {row["Name_Cap"]} 0 "false" 1 "true"', axis=1)
Cog["encoded"] = Cog.apply(lambda row: f'encode {row["Name_Cap"]}, gen ({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Cog["drop"] = Cog.apply(lambda row: f'drop {row["Name_Cap"]}', axis=1)
Cog["checked_unchecked"] = Cog.apply(lambda row: f'label values {row["Name"]} checkbox_label', axis=1)
Cog['combined_master_columns'] = Cog.apply(lambda row: f'{row["rename"]}\n\n{row["label"]}\n\n{row["encoded"]}\n\n{row["drop"]}', axis=1)

Cog_file = 'Recode_Files/Cog_code_checks.txt'
with open(Cog_file, 'w') as f:
    for code_line in Cog['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Cog_file = 'Recode_Files/Cog_code_checks_renamed.txt'
with open(Cog_file, 'w') as f:
    # Write the additional text at the top of the file
    f.write('label define checkbox_label 0 "unchecked" 1 "checked", replace\n\n')

    # Write each code line from the "combined_master_columns" column
    for code_line in Cog['checked_unchecked']:
        f.write(f"{code_line}\n\n")

Cog.head()

,Field_Type,Name,Special_Info,Name_Cap,rename,label,encoded,drop,checked_unchecked,combined_master_columns
0,Checkbox,c_33_a,NaN,C_33_A,rename c_33_a (C_33_A),"label define C_33_A 0 ""false"" 1 ""true""","encode C_33_A, gen (c_33_a) label (C_33_A)",drop C_33_A,label values c_33_a checkbox_label,rename c_33_a (C_33_A)\n\nlabel define C_33_A ...
1,Checkbox,c_33_1,NaN,C_33_1,rename c_33_1 (C_33_1),"label define C_33_1 0 ""false"" 1 ""true""","encode C_33_1, gen (c_33_1) label (C_33_1)",drop C_33_1,label values c_33_1 checkbox_label,rename c_33_1 (C_33_1)\n\nlabel define C_33_1 ...
2,Checkbox,c_33_2,NaN,C_33_2,rename c_33_2 (C_33_2),"label define C_33_2 0 ""false"" 1 ""true""","encode C_33_2, gen (c_33_2) label (C_33_2)",drop C_33_2,label values c_33_2 checkbox_label,rename c_33_2 (C_33_2)\n\nlabel define C_33_2 ...
3,Checkbox,c_33_3,NaN,C_33_3,rename c_33_3 (C_33_3),"label define C_33_3 0 ""false"" 1 ""true""","encode C_33_3, gen (c_33_3) label (C_33_3)",drop C_33_3,label values c_33_3 checkbox_label,rename c_33_3 (C_33_3)\n\nlabel define C_33_3 ...
4,Checkbox,c_33_4,NaN,C_33_4,rename c_33_4 (C_33_4),"label define C_33_4 0 ""false"" 1 ""true""","encode C_33_4, gen (c_33_4) label (C_33_4)",drop C_33_4,label values c_33_4 checkbox_label,rename c_33_4 (C_33_4)\n\nlabel define C_33_4 ...


In [ ]:
Cog = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Cognitive")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Text') | (df['Field_Type'] == 'Legal Values'), ['Field_Type', 'Name']]
    .reset_index(drop=True)
)

Cog['Name'] = Cog['Name'].str.lower()

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Cog[col] = Cog[col].astype(str).str.strip()

Cog['Name_Cap'] = Cog['Name'].str.upper()

Cog["capture"] = Cog.apply(lambda row: f'capture confirm numeric variable {row["Name"]}\nif !_rc'+"{", axis=1)
Cog["convert"] = Cog.apply(lambda row: f'     tostring {row["Name"]}, replace\n'+"}", axis=1)

Cog['combined_master_columns'] = Cog.apply(lambda row: f'{row["capture"]}\n{row["convert"]}', axis=1)

Cog_file = 'Recode_Files/Cog_text.txt'
with open(Cog_file, 'w') as f:
    for code_line in Cog['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Cog.head()


,Field_Type,Name,Name_Cap,capture,convert,combined_master_columns
0,Text,c_interid,C_INTERID,capture confirm numeric variable c_interid\nif...,"tostring c_interid, replace\n}",capture confirm numeric variable c_interid\nif...
1,Legal Values,c_particid2,C_PARTICID2,capture confirm numeric variable c_particid2\n...,"tostring c_particid2, replace\n}",capture confirm numeric variable c_particid2\n...
2,Text,c_deviceid1,C_DEVICEID1,capture confirm numeric variable c_deviceid1\n...,"tostring c_deviceid1, replace\n}",capture confirm numeric variable c_deviceid1\n...
3,Text,c_20,C_20,capture confirm numeric variable c_20\nif !_rc{,"tostring c_20, replace\n}",capture confirm numeric variable c_20\nif !_rc...
4,Legal Values,c_26_1,C_26_1,capture confirm numeric variable c_26_1\nif !_rc{,"tostring c_26_1, replace\n}",capture confirm numeric variable c_26_1\nif !_...


In [ ]:
Cog = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Cognitive") #everything except label/title, group, Time
    .loc[lambda df: ~df['Field_Type'].isin(['Label/Title', 'Group', 'Time','Date'])]
    .reset_index(drop=True)
)

Cog['Name'] = Cog['Name'].str.lower()
Cog['Prompt'] = Cog['Prompt'].replace(r'[\'"]', '', regex=True)

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Cog[col] = Cog[col].astype(str).str.strip()

Cog["label"] = Cog.apply(lambda row: f'label variable {row["Name"]} "{row["Prompt"]}"', axis=1)

Cog_file = 'Recode_Files/Cog_variable_labels.txt'
with open(Cog_file, 'w') as f:
    for code_line in Cog['label']:
        f.write(f"{code_line}\n\n")

name_list = ' '.join(Cog['Name'])

# Add the word 'order' at the beginning of the string
name_list = 'order ' + name_list

# Write the string to a text file
with open('Recode_Files/Cog_Names.txt', 'w') as f:
    f.write(name_list)

Cog.head()


,Page Position,Page Name,Tab Index,Prompt,Field_Type,Name,Variable Type,Format,Special_Info,label
0,0,INTRO,3,NUMERO DEL ENTREVISTADOR,Number,c_interid,Number,NaN,Required,"label variable c_interid ""NUMERO DEL ENTREVIST..."
1,0,INTRO,4,NÚMERO DE LA CASA,Number,c_houseid,Number,NaN,Read Only,"label variable c_houseid ""NÚMERO DE LA CASA"""
2,0,INTRO,5,NUMERO DEL CLUSTER:,Number,c_clustid,Number,NaN,Read Only,"label variable c_clustid ""NUMERO DEL CLUSTER:"""
3,0,INTRO,6,NUMERO DEL PARTICIPANTE,Number,c_particid,Number,NaN,Read Only,"label variable c_particid ""NUMERO DEL PARTICIP..."
4,0,INTRO,7,Pais,Number,c_country,Number,NaN,Read Only,"label variable c_country ""Pais"""


Household

In [ ]:
Household = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Household")  # replace with your file path
    .loc[lambda df: df['Field_Type'] == 'Option', ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Household['Special_Info'] = Household['Special_Info'].str.lower()
Household['Name'] = Household['Name'].str.lower()
Household['Special_Info'] = Household['Special_Info'].str.replace(' required', '', case=False)

columns_to_process = ['Field_Type', 'Name', 'Special_Info']

for col in columns_to_process:
    Household[col] = Household[col].astype(str).str.strip()

Household['Name_Cap'] = Household['Name'].str.upper()

split_special_info = Household['Special_Info'].str.split(',', expand=True)

split_special_info.columns = range(1, len(split_special_info.columns)+1)
split_special_info = split_special_info.applymap(lambda x: x.strip() if x is not None else None)

Household = pd.concat([Household, split_special_info], axis=1)

Household = Household.replace({None: np.nan})

for i in range(1, 9):
    Household[f'new_column{i}'] = Household.apply(lambda row: str(f'{Household.columns[i+3]} "{row[i]}" '), axis=1).astype(str)

for i in range(1, 9):
    Household[f'new_column{i}'] = (
       Household[f'new_column{i}']
        .apply(lambda x: '8 "no responde"' if 'no responde' in str(x).lower() else x)
        .apply(lambda x: '9 "no sabe"' if 'no sabe' in str(x).lower() else x)
        .apply(lambda x: '0 "no"' if '"no"' in str(x).lower() else x)
        .apply(lambda x: '1 "si"' if '"si"' in str(x).lower() else x)
    )

Household['new_combined_column'] = 'label define ' + Household['Name_Cap'].astype(str) + ' .a"."' + ' '

for i in range(1, 9):
    Household['new_combined_column'] += Household.apply(lambda row: row[f"new_column{i}"] if 'nan' not in str(row[f"new_column{i}"]).lower() else '', axis=1)

Household["capture"] = Household.apply(lambda row: f'capture confirm numeric variable {row["Name_Cap"]}\nif !_rc'+"{", axis=1)
Household["convert"] = Household.apply(lambda row: f'     tostring {row["Name_Cap"]}, replace\n'+"}", axis=1)
Household['encode_column'] = Household.apply(lambda row: f'encode {row["Name_Cap"]}, gen({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Household["rename"] = Household.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)

Household['combined_master_columns'] = Household.apply(lambda row: f'{row["rename"]}\n\n{row["capture"]}\n{row["convert"]}\n\n{row["new_combined_column"]}\n\n{row["encode_column"]}', axis=1)

Household_file = 'Recode_Files/Household_code.txt'
with open(Household_file, 'w') as f:
    for code_line in Household['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Household = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Household")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Checkbox') | (df['Field_Type'] == 'Checkbox'), ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Household['Name_Cap'] = Household['Name']
Household['Name'] = Household['Name'].str.lower()

Household["rename"] = Household.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)
Household["label"] = Household.apply(lambda row: f'label define {row["Name_Cap"]} 0 "False" 1 "True"', axis=1)
Household["encoded"] = Household.apply(lambda row: f'encode {row["Name_Cap"]}, gen ({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Household["drop"] = Household.apply(lambda row: f'drop {row["Name_Cap"]}', axis=1)
Household['combined_master_columns'] = Household.apply(lambda row: f'{row["rename"]}\n\n{row["label"]}\n\n{row["encoded"]}\n\n{row["drop"]}', axis=1)

Household_file = 'Recode_Files/Household_code_checks.txt'
with open(Household_file, 'w') as f:
    for code_line in Household['combined_master_columns']:
        f.write(f"{code_line}\n\n")


Household.head()

,Field_Type,Name,Special_Info,Name_Cap,rename,label,encoded,drop,combined_master_columns


In [ ]:
Household = (
    pd.read_excel("CADAS_DD_V3.xlsx", engine='openpyxl', sheet_name="Household")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Text') | (df['Field_Type'] == 'Legal Values'), ['Field_Type', 'Name']]
    .reset_index(drop=True)
)

Household['Name'] = Household['Name'].str.lower()

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Household[col] = Household[col].astype(str).str.strip()

Household['Name_Cap'] = Household['Name'].str.upper()

Household["capture"] = Household.apply(lambda row: f'capture confirm numeric variable {row["Name"]}\nif !_rc'+"{", axis=1)
Household["convert"] = Household.apply(lambda row: f'     tostring {row["Name"]}, replace\n'+"}", axis=1)

Household['combined_master_columns'] = Household.apply(lambda row: f'{row["capture"]}\n{row["convert"]}', axis=1)

Household_file = 'Recode_Files/Household_text.txt'
with open(Household_file, 'w') as f:
    for code_line in Household['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Household.head()


,Field_Type,Name,Name_Cap,capture,convert,combined_master_columns
0,Text,h_deviceid1,H_DEVICEID1,capture confirm numeric variable h_deviceid1\n...,"tostring h_deviceid1, replace\n}",capture confirm numeric variable h_deviceid1\n...
1,Text,h_deviceid2,H_DEVICEID2,capture confirm numeric variable h_deviceid2\n...,"tostring h_deviceid2, replace\n}",capture confirm numeric variable h_deviceid2\n...


In [ ]:
Household = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Household") #everything except label/title, group, Time
    .loc[lambda df: ~df['Field_Type'].isin(['Label/Title', 'Group', 'Time','Date'])]
    .reset_index(drop=True)
)

Household['Name'] = Household['Name'].str.lower()
Household['Prompt'] = Household['Prompt'].replace(r'[\'"]', '', regex=True)

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Household[col] = Household[col].astype(str).str.strip()

Household["label"] = Household.apply(lambda row: f'label variable {row["Name"]} "{row["Prompt"]}"', axis=1)

Household_file = 'Recode_Files/Household_variable_labels.txt'
with open(Household_file, 'w') as f:
    for code_line in Household['label']:
        f.write(f"{code_line}\n\n")

name_list = ' '.join(Household['Name'])

# Add the word 'order' at the beginning of the string
name_list = 'order ' + name_list

# Write the string to a text file
with open('Recode_Files/Household_Names.txt', 'w') as f:
    f.write(name_list)

Household.head()


,Page Position,Page Name,Tab Index,Prompt,Field_Type,Name,Variable Type,Format,Special_Info,label
0,0,Intro and Background,5,Número del entrevistador:,Number,h_interid,Number,NaN,Required,"label variable h_interid ""Número del entrevist..."
1,0,Intro and Background,6,Número de la casa:,Number,h_houseid,Number,NaN,Read Only,"label variable h_houseid ""Número de la casa:"""
2,0,Intro and Background,7,Número del cluster,Number,h_clustid,Number,NaN,Read Only,"label variable h_clustid ""Número del cluster"""
3,0,Intro and Background,8,Número de la casa:,Number,h_houseid2,Number,NaN,Required,"label variable h_houseid2 ""Número de la casa:"""
4,0,Intro and Background,9,Número del cluster,Number,h_conglid2,Number,NaN,Required,"label variable h_conglid2 ""Número del cluster"""


Informant

In [ ]:
Infor = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Informant")  # replace with your file path
    .loc[lambda df: df['Field_Type'] == 'Option', ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Infor['Special_Info'] = Infor['Special_Info'].str.lower()
Infor['Name'] = Infor['Name'].str.lower()
Infor['Special_Info'] = Infor['Special_Info'].str.replace(' required', '', case=False)

columns_to_process = ['Field_Type', 'Name', 'Special_Info']

for col in columns_to_process:
    Infor[col] = Infor[col].astype(str).str.strip()

Infor['Name_Cap'] = Infor['Name'].str.upper()

split_special_info = Infor['Special_Info'].str.split(',', expand=True)

split_special_info.columns = range(1, len(split_special_info.columns)+1)
split_special_info = split_special_info.applymap(lambda x: x.strip() if x is not None else None)

Infor = pd.concat([Infor, split_special_info], axis=1)

Infor = Infor.replace({None: np.nan})

for i in range(1, 10):
    Infor[f'new_column{i}'] = Infor.apply(lambda row: str(f'{Infor.columns[i+3]} "{row[i]}" '), axis=1).astype(str)

for i in range(1, 10):
    Infor[f'new_column{i}'] = (
       Infor[f'new_column{i}']
        .apply(lambda x: '8 "no responde"' if 'no responde' in str(x).lower() else x)
        .apply(lambda x: '9 "no sabe"' if 'no sabe' in str(x).lower() else x)
        .apply(lambda x: '0 "no"' if '"no"' in str(x).lower() else x)
        .apply(lambda x: '1 "si"' if '"si"' in str(x).lower() else x)
    )

Infor['new_combined_column'] = 'label define ' + Infor['Name_Cap'].astype(str) + ' .a"."' + ' '

for i in range(1, 10):
    Infor['new_combined_column'] += Infor.apply(lambda row: row[f"new_column{i}"] if 'nan' not in str(row[f"new_column{i}"]).lower() else '', axis=1)

Infor["capture"] = Infor.apply(lambda row: f'capture confirm numeric variable {row["Name_Cap"]}\nif !_rc'+"{", axis=1)
Infor["convert"] = Infor.apply(lambda row: f'     tostring {row["Name_Cap"]}, replace\n'+"}", axis=1)
Infor['encode_column'] = Infor.apply(lambda row: f'encode {row["Name_Cap"]}, gen({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Infor["rename"] = Infor.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)

Infor['combined_master_columns'] = Infor.apply(lambda row: f'{row["rename"]}\n\n{row["capture"]}\n{row["convert"]}\n\n{row["new_combined_column"]}\n\n{row["encode_column"]}', axis=1)

Infor_file = 'Recode_Files/Infor_code.txt'
with open(Infor_file, 'w') as f:
    for code_line in Infor['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Infor = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Informant")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Checkbox') | (df['Field_Type'] == 'Checkbox'), ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Infor['Name_Cap'] = Infor['Name']
Infor['Name'] = Infor['Name'].str.lower()

Infor["rename"] = Infor.apply(lambda row: f'rename {row["Name"]} ({row["Name_Cap"]})', axis=1)
Infor["label"] = Infor.apply(lambda row: f'label define {row["Name_Cap"]} 0 "False" 1 "True"', axis=1)
Infor["encoded"] = Infor.apply(lambda row: f'encode {row["Name_Cap"]}, gen ({row["Name"]}) label ({row["Name_Cap"]})', axis=1)
Infor["drop"] = Infor.apply(lambda row: f'drop {row["Name_Cap"]}', axis=1)
Infor['combined_master_columns'] = Infor.apply(lambda row: f'{row["rename"]}\n\n{row["label"]}\n\n{row["encoded"]}\n\n{row["drop"]}', axis=1)

Infor_file = 'Recode_Files/Infor_code_checks.txt'
with open(Infor_file, 'w') as f:
    for code_line in Infor['combined_master_columns']:
        f.write(f"{code_line}\n\n")


Infor.head()

,Field_Type,Name,Special_Info,Name_Cap,rename,label,encoded,drop,combined_master_columns


In [ ]:
Infor = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Informant")  # replace with your file path
    .loc[lambda df: (df['Field_Type'] == 'Text') | (df['Field_Type'] == 'Legal Values'), ['Field_Type', 'Name', 'Special_Info']]
    .reset_index(drop=True)
)

Infor['Name'] = Infor['Name'].str.lower()

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Infor[col] = Infor[col].astype(str).str.strip()

Infor['Name_Cap'] = Infor['Name'].str.upper()

Infor["capture"] = Infor.apply(lambda row: f'capture confirm numeric variable {row["Name"]}\nif !_rc'+"{", axis=1)
Infor["convert"] = Infor.apply(lambda row: f'     tostring {row["Name"]}, replace\n'+"}", axis=1)

Infor['combined_master_columns'] = Infor.apply(lambda row: f'{row["capture"]}\n{row["convert"]}', axis=1)

Infor_file = 'Recode_Files/Infor_text.txt'
with open(Infor_file, 'w') as f:
    for code_line in Infor['combined_master_columns']:
        f.write(f"{code_line}\n\n")

Infor.head()


,Field_Type,Name,Special_Info,Name_Cap,capture,convert,combined_master_columns
0,Legal Values,i_particid2,Required,I_PARTICID2,capture confirm numeric variable i_particid2\n...,"tostring i_particid2, replace\n}",capture confirm numeric variable i_particid2\n...
1,Text,i_deviceid1,Read Only,I_DEVICEID1,capture confirm numeric variable i_deviceid1\n...,"tostring i_deviceid1, replace\n}",capture confirm numeric variable i_deviceid1\n...
2,Text,i_deviceid2,Read Only,I_DEVICEID2,capture confirm numeric variable i_deviceid2\n...,"tostring i_deviceid2, replace\n}",capture confirm numeric variable i_deviceid2\n...
3,Text,i_deviceid2,Read Only,I_DEVICEID2,capture confirm numeric variable i_deviceid2\n...,"tostring i_deviceid2, replace\n}",capture confirm numeric variable i_deviceid2\n...


In [ ]:
Infor = (
    pd.read_excel("CADAS_DD_V4.xlsx", engine='openpyxl', sheet_name="Informant") #everything except label/title, group, Time
    .loc[lambda df: ~df['Field_Type'].isin(['Label/Title', 'Group', 'Time','Date'])]
    .reset_index(drop=True)
)

Infor['Name'] = Infor['Name'].str.lower()
Infor['Prompt'] = Infor['Prompt'].replace(r'[\'"]', '', regex=True)

columns_to_process = ['Field_Type', 'Name']

for col in columns_to_process:
    Infor[col] = Infor[col].astype(str).str.strip()

Infor["label"] = Infor.apply(lambda row: f'label variable {row["Name"]} "{row["Prompt"]}"', axis=1)

Infor_file = 'Recode_Files/Infor_variable_labels.txt'
with open(Infor_file, 'w') as f:
    for code_line in Infor['label']:
        f.write(f"{code_line}\n\n")

name_list = ' '.join(Infor['Name'])

# Add the word 'order' at the beginning of the string
name_list = 'order ' + name_list

# Write the string to a text file
with open('Recode_Files/Infor_Names.txt', 'w') as f:
    f.write(name_list)

Infor.head()

,Page Position,Page Name,Tab Index,Prompt,Field_Type,Name,Variable Type,Format,Special_Info,label
0,0,Intro and Background,6,Número del entrevistador:,Number,i_interid,Number,NaN,Required,"label variable i_interid ""Número del entrevist..."
1,0,Intro and Background,7,Número del cluster:,Number,i_clustid,Number,NaN,Read Only,"label variable i_clustid ""Número del cluster:"""
2,0,Intro and Background,8,Número de la casa:,Number,i_houseid,Number,NaN,Read Only,"label variable i_houseid ""Número de la casa:"""
3,0,Intro and Background,9,Número del participante:,Number,i_particid,Number,NaN,Read Only,"label variable i_particid ""Número del particip..."
4,0,Intro and Background,10,Pais,Number,i_country,Number,NaN,Read Only,"label variable i_country ""Pais"""
